In [ ]:
from ipywidgets import Layout
from IPython.display import Javascript, HTML
import ipywidgets as widgets

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

path_to_label = widgets.Text(
    placeholder='Label path here',
    description='Label:',
    disabled=False,
    layout=Layout(width='100%')
)
path_to_image = widgets.Text(
    placeholder='Image path here',
    description='Image:',
    disabled=False,
    layout=Layout(width='100%')
)

In [ ]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
display(path_to_label, path_to_image)
button = widgets.Button(description="Create next input")
button.on_click(run_all)
display(button)

In [ ]:
import json 
from PIL import Image, ImageDraw
from IPython.display import display
from shapely import wkt

In [ ]:
# Opening and loading polygons from label json 
path_to_label_value = path_to_label.value
path_to_image_value = path_to_image.value

with open(path_to_label_value, 'rb') as image_json_file:
    image_json = json.load(image_json_file)

In [ ]:
coords = image_json['features']['xy']
wkt_polygons = []

for coord in coords:
    if 'subtype' in coord['properties']:
        damage = coord['properties']['subtype']
    else:
        damage = 'no-damage'
    wkt_polygons.append((damage, coord['wkt']))
    
polygons = []

for damage, swkt in wkt_polygons:
    polygons.append((damage, wkt.loads(swkt)))

In [ ]:
# Loading image
img = Image.open(path_to_image_value)

In [ ]:
draw = ImageDraw.Draw(img, 'RGBA')

damage_dict = {
    "no-damage": (0, 255, 0, 100),
    "minor-damage": (0, 0, 255, 125),
    "major-damage": (255, 69, 0, 125),
    "destroyed": (255, 0, 0, 125),
    "un-classified": (255, 255, 255, 125)
}

for damage, polygon in polygons:
    x,y = polygon.exterior.coords.xy
    coords = list(zip(x,y))
    draw.polygon(coords, damage_dict[damage])

del draw

display(img)